In [2]:
!pip install librosa
!pip install tensorflow
!pip install numpy==1.23.5

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Tensor flow does not work with Numpy 2.0 onward so make sure that 

In [3]:
#Imports
from pydub import AudioSegment as audio
import os
import numpy as np
import librosa
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import re 

1.23.5


/Users/ryanbirmingham/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Import explanations
    - AudioSegment
    - os
        Used for getting the paths for the audio files and saving them to new folders
    - librosa
        Package for audio analysis

In [5]:
unproccessed_data_dir = os.path.abspath('') + "/Unproccessed_Wav_Files"

Steps
- go through all files in the unprocessed wav files
- break up all files so they are split by 1 second
- use regex to get the file name minus the .wav at the end to make new 1 second wav files
- save each of the files into the processed wav folder

In [8]:
fpath = f"{os.path.abspath('')}/Processed_Wav_Files/"
for filename in os.listdir(unproccessed_data_dir):
    if filename.endswith(".wav"):
        hearing = audio.from_wav(os.path.join(unproccessed_data_dir,filename))
        speaker = filename.removesuffix(".wav")
        #name of congress person for processed_wav_files folder
        congress_name = speaker[:re.search(r"\d+",speaker).start()] 
        #start duration in milliseconds for audio recording start
        t1 = 0
        #end duration in milliseconds for audio recording start
        t2 = 1000
        #second of the clip
        second = 1
        total_time = hearing.duration_seconds*1000
        while t2 <= total_time:
            new_audio = hearing[t1:t2]
            try:
                os.makedirs(f"{fpath}{congress_name}")
                new_audio.export(f"{fpath}/{congress_name}/{speaker}_{second}.wav", format = "wav");
            except:
                new_audio.export(f"{fpath}/{congress_name}/{speaker}_{second}.wav", format = "wav");
            t1 += 1000
            t2 += 1000
            second += 1

stephenCohen
michaelQuigley
addisonWilson
christineSmith
michaelLawler
samuelgraves
victoriaSpartz
kevinKiley
addisonWilson
richardlarsen
addisonWilson
stephenCohen
addisonWilson
stephenCohen
sheldonWhitehouse
stephenCohen
